In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

## 读入数据

2022.01.08

- 因为xarray会将读入nc文件含有units = days 的数据识别为日期，decode_times 需要关闭，从而读入dtype为float64的 天数，而非一个时间dtype

2022.02.14

将分析时段调整到MJJA，并且调整色板

In [ ]:
ds_in = {}
for mod_name in ['obs','vr', 'rcm']:
    ds_in[mod_name] = {}
    ds_in[mod_name]['mjja'] = xr.open_dataset("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/extreme/extreme_"+mod_name+"_5-8.nc", decode_times = False)
vars_names = list(ds_in[mod_name]['mjja'].variables)[3:]
vars_names

## 计算平均

In [ ]:
ds_mean = {}
for mod_name in ['obs','vr','rcm']:
    ds_mean[mod_name] = {}
    for season in ['mjja']:
        ds_mean[mod_name][season] = ds_in[mod_name][season].mean(dim = 'time')

lon = ds_mean['obs']['mjja']['CDD'].lon.values
lat = ds_mean['obs']['mjja']['CDD'].lat.values

In [ ]:
import skill_metrics as sm
metrics_ds_mean = {}

for mod_name in ['vr','rcm']:
    metrics_ds_mean[mod_name] = {}
    for var_name in ['RX5day', 'SDII', 'CWD', 'CDD']:
        metrics_ds_mean[mod_name][var_name] = pd.DataFrame(index = [mod_name], columns=['Bias','RMSE','Corr.']) 
        temp_obs = ds_mean['obs']['mjja'][var_name].values.ravel()
        temp_mod = ds_mean[mod_name]['mjja'][var_name].values.ravel()

        # remove nan values
        not_nan_index =( ( ~np.isnan(ds_mean['obs']['mjja'][var_name].values.ravel()) ) & \
                         ( ~np.isnan(ds_mean['vr']['mjja'][var_name].values.ravel()) ) & \
                         ( ~np.isnan(ds_mean['rcm']['mjja'][var_name].values.ravel()) ) )

        temp_obs = temp_obs[not_nan_index]
        temp_mod = temp_mod[not_nan_index]

        metrics_ds_mean[mod_name][var_name].loc[mod_name]['Bias'] = sm.bias(temp_mod, temp_obs)
        metrics_ds_mean[mod_name][var_name].loc[mod_name]['RMSE'] = sm.rmsd(temp_mod, temp_obs)
        metrics_ds_mean[mod_name][var_name].loc[mod_name]['Corr.'] = np.corrcoef(temp_mod, temp_obs)[0,1]

        # # 对style进行调整，用于后续展示表格
        # # https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    # metrics_ds_mean[mod_name] = metrics_ds_mean[mod_name].style.format("{:,.3f}")

## 绘图部分
2022.01.09

为了方便绘图，更换了版本，用来传入控制左边标题的参数 proplot 0.6.4-py_0 --> 0.9.5-pyhd8ed1ab_1

### 绘图准备部分
包含写为函数的地图的绘制，以等距的spacing norm

In [ ]:
## 地图绘制预备数据
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps as cmaps
from matplotlib.font_manager import FontProperties

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    ##---- 直接绘图，从边界文件添加
    # for ax_ind in axs:
    # for line in borders:
    #     axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    #     axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        # shapefile数据下载的位置：
    # http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = "gbk")
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)


### 绘图测试部分
因为子图过多，此处测试色板 norm ticks等要素

In [ ]:
# ticks = np.linspace(0,100,11)  # CDD CWD
# ticks = np.linspace(0,10000,int(10000 / 250 + 1)) # RX5day
# ticks = np.concatenate( (np.linspace(0,4000,21),np.linspace(5000,12000,8)) , axis = 0) # RX5day
# ticks =np.concatenate( (np.linspace(0,40,41),np.linspace(50,120,8)) , axis = 0)
# ticks =  np.concatenate( (np.linspace(0,40,21),np.linspace(50,100,6)) , axis = 0)
ticks_diff = np.arange(-10,10,11)
# cmap = cmaps.MPL_s3pcpn_l
cmap = cmaps.BkBlAqGrYeOrReViWh200
fig, axs = pplt.subplots(nrows = 1,ncols = 3)
var = 'CDD'
mcontourf = axs[0].contourf(ds_mean['obs']['mjja'][var].values, cmap = cmap, levels = ticks, norm = 'segmented')
mcontourf = axs[1].contourf(ds_mean['vr']['mjja'][var].values - ds_mean['obs']['mjja'][var].values, cmap = cmap, levels = ticks_diff, norm = 'div')
mcontourf = axs[2].contourf(ds_mean['rcm']['mjja'][var].values - ds_mean['obs']['mjja'][var].values, cmap = cmap, levels = ticks_diff, norm = 'div')
axs.format(title = var)
# axs[2].colorbar(mcontourf, ticks = ticks[::2], length = 0.9, extend = 'max',title=color_bar_title[var])

### 完整绘图部分

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import collections
# ----- get filter vars coords-----

#----- change font -----
pplt.rc['font.serif'] = 'Arial'

#----- create plot -----
# fig, axs = plot.subplots(ncols = 3, nrows = 4, proj=('cyl'), share = True)
# fig, axs = plot.subplots(ncols=3 ,nrows=4, proj=('cyl'), width = '190mm',height = '115mm', share = True)
fig, axs = plot.subplots(ncols=3 ,nrows=4, proj=('cyl'), width = '190mm',height = '180mm', share = True)
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
border_plot(axs)
    
#----- colorbar ticks 统一设置 -----
cmap_dict = {}

# cmap_dict['mean'] = cmaps.WhiteBlueGreenYellowRed
# cmap_dict['mean_diff'] = cmaps.cmp_flux
cmap_dict['RX5day'] = cmaps.WhiteBlueGreenYellowRed
cmap_dict['SDII'] = cmaps.WhiteBlueGreenYellowRed
cmap_dict['CDD'] = cmaps.BkBlAqGrYeOrReViWh200
# cmap_dict['CWD'] = cmaps.NCV_bright_r
cmap_dict['CWD'] = cmaps.WhBlGrYeRe

ticks_dict = {}
# ticks_dict['mean']      = np.concatenate((np.linspace(0,10,21), np.linspace(11,20,10)), axis=0)
# ticks_dict['mean_diff'] = np.linspace(-5, 15, 21)
ticks_dict['RX5day']    = np.concatenate( (np.linspace(0,300,11),np.linspace(300,600,6)[1:]) , axis = 0) # RX5day
ticks_dict['SDII']      = np.concatenate( (np.linspace(0,30,11),[40]) , axis = 0)
ticks_dict['CDD']       = np.concatenate( (np.linspace(0,40,9),np.linspace(50,100,6)) , axis = 0)
ticks_dict['CWD']       = np.concatenate( (np.linspace(0,40,9),np.linspace(50,100,6)) , axis = 0)
# ticks_dict['CDD']       = np.linspace(0, 100, 11)
# ticks_dict['CWD']       = np.linspace(0, 100, 11)

# ------ axs plots -----
def nested_dict(): # 用于进行绘制的多重dict的设置
    return collections.defaultdict(nested_dict)
mcontourf_dict = nested_dict()

plot_ind = 0
# 绘制极端指数
for var in vars_names:    
    for season in ['mjja']:
        for mod_name in ['obs','vr','rcm']:
            # 使用了均分的norm，处理不等距的ticks
            mcontourf_temp = axs[plot_ind].contourf(lon, lat, ds_mean[mod_name][season][var].values,\
            cmap=cmap_dict[var], levels = ticks_dict[var], norm = 'segmented', extend = 'both')
            mcontourf_dict[season][mod_name][var] = mcontourf_temp # 合并所有绘图obj到一个deep dict之中
            plot_ind = plot_ind + 1

#----- add color bar-----
color_bar_title = {}
# color_bar_title['RX5day'] = '5-day precipitation amount ' + r"$[mm/d]$"
# color_bar_title['SDII']   = 'precipitation during wet days ' + r"$[mm/d]$"
# color_bar_title['CDD']    = 'consecutive dry days ' + "[days]"
# color_bar_title['CWD']    = 'consecutive wet days ' + "[days]"

color_bar_title['RX5day'] = r"$[mm/d]$"
color_bar_title['SDII']   = r"$[mm/d]$"
color_bar_title['CDD']    = r"[days]"
color_bar_title['CWD']    = r"[days]"

# 极端指数 colorbar
plot_ind = 2
# colorbar_locations = [(1,2), (1,5), (2,2), (2,5)]
for var in vars_names:
    axs[plot_ind].colorbar(mcontourf_dict['mjja']['obs'][var], loc='r', width=0.1, extend = 'both',length = 0.96,\
    ticklabelsize=5,labelsize = 7,ticks=ticks_dict[var][::], title=color_bar_title[var], pad = 0.2)
    plot_ind = plot_ind + 3

# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
labels = False,
longrid  = True,
latgrid  = True,
#-----GEO axis-----
lonlim=(70, 140), latlim=(5, 60),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(5,70+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(5,70+2,2),
#-----line label-----
# linewidth = 0.5,
# suptitle="precipitation",

toplabels=('CN05.1_CMORPH', 'VR', 'RCM'),
leftlabels=('RX5day', 'SDII','CDD','CWD'),
# rc_kw = {'leftlabel.rotation':90.}

)

# ----- 合并子图之后控制边界的labels绘制 ----- #
axs[:-1,0].format(labels = True, lonlabels = False, latlabels = True)
axs[-1,1:].format(labels = True, lonlabels = True, latlabels = False)
axs[-1,0].format(labels = True, lonlabels = True, latlabels = True)

# ----- add table on axis -----
# 参考回答：https://stackoverflow.com/questions/54150557/how-to-show-dataframe-index-name-on-a-matplotlib-table
# 使用bbox调整表格的大小
plot_ind = 0
    # metrics_ds_mean[mod_name] = {}
for var_name in ['RX5day', 'SDII', 'CDD', 'CWD']:
    for mod_name in ['obs','vr','rcm']:
        if (mod_name == 'obs'):
            plot_ind +=1
            continue
        
        # plot_table = metrics_ds_mean[mod_name][var_name].applymap("{:,.2f}".format)
        plot_table = metrics_ds_mean[mod_name][var_name]
        table = axs[plot_ind].table(cellText=np.matrix.round(plot_table.values.astype('float'),2),\
                    colLabels=plot_table.columns, loc = 'center',colColours=['gainsboro'] * 3,\
                    colWidths = [0.13]*3, bbox = [0.09, .82, 0.35, 0.15], zorder = 10)
        table.auto_set_font_size(False) # 关闭自动网格字体大小设置
        table.set_fontsize(5.) # 设置数字字体大小
        # table.set_fontweight('roman')
                    #  colColours=['gainsboro'] * len(plot_table), colLabels=plot_table.columns, loc='center',
                    #  colWidths= [0.12]*(len(plot_table.columns)))
        for (row, col), cell in table.get_celld().items():
            if ((row == 0)):
                cell.set_text_props(fontproperties=FontProperties(weight='bold', size = 5)) # 设置collabels字体、并且灰色网格，加粗
        plot_ind +=1

#----- save figure -----
fig.patch.set_facecolor('white')
fig.savefig('./output_pic/pre_extreme_2022.05.02.jpg',dpi = 600, facecolor= "white", bbox_inches='tight')